In [ ]:
import os
import json
from datetime import datetime

import numpy as np
import pandas as pd
import joblib


In [46]:
def clean_claims_data(df: pd.DataFrame) -> pd.DataFrame:
    """Mirror the training-time claims cleaning pipeline."""
    df_clean = df.copy()

    # 1. Standardise column names
    df_clean.columns = df_clean.columns.str.strip().str.lower().str.replace(' ', '_')

    # 2. Parse date columns
    date_columns = [
        'paid_date', 'incurred_date', 'admission_date', 'discharge_date',
        'contract_start_date', 'contract_end_date'
    ]
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

    # 3. Remove invalid rows
    if 'claim_amount' in df_clean.columns:
        df_clean = df_clean[df_clean['claim_amount'] > 0]
    if 'paid_date' in df_clean.columns:
        df_clean = df_clean[df_clean['paid_date'] <= pd.Timestamp.now()]
    if 'incurred_date' in df_clean.columns and 'paid_date' in df_clean.columns:
        df_clean = df_clean[df_clean['incurred_date'] <= df_clean['paid_date']]

    # 4. Fill missing categorical fields
    if 'treatment_type' in df_clean.columns:
        df_clean['treatment_type'].fillna('Unknown', inplace=True)
    if 'provider_type' in df_clean.columns:
        df_clean['provider_type'].fillna('Unknown', inplace=True)
    if 'treatment_location' in df_clean.columns:
        mode_val = df_clean['treatment_location'].mode()
        df_clean['treatment_location'].fillna(mode_val[0] if not mode_val.empty else 'Unknown', inplace=True)
    if 'condition_category' in df_clean.columns:
        df_clean['condition_category'].fillna('Unspecified', inplace=True)
    if 'admission_date' in df_clean.columns and 'discharge_date' in df_clean.columns:
        mask = df_clean['admission_date'].notna() & df_clean['discharge_date'].isna()
        df_clean.loc[mask, 'discharge_date'] = df_clean.loc[mask, 'admission_date']

    # 5. Standardise categoricals
    if 'claimant_gender' in df_clean.columns:
        gender_mapping = {
            'M': 'Male', 'F': 'Female', 'O': 'Other',
            'male': 'Male', 'female': 'Female', 'other': 'Other',
            'MALE': 'Male', 'FEMALE': 'Female', 'OTHER': 'Other'
        }
        df_clean['claimant_gender'] = df_clean['claimant_gender'].map(
            lambda x: gender_mapping.get(x, x) if pd.notna(x) else 'Unknown'
        )
    if 'status_of_member' in df_clean.columns:
        df_clean['status_of_member'] = df_clean['status_of_member'].astype(str).str.strip().str.title()

    # 6. Flag outliers (retain them for modelling)
    if 'claim_amount' in df_clean.columns:
        q1 = df_clean['claim_amount'].quantile(0.25)
        q3 = df_clean['claim_amount'].quantile(0.75)
        iqr = q3 - q1
        df_clean['is_outlier'] = (
            (df_clean['claim_amount'] < (q1 - 3 * iqr)) |
            (df_clean['claim_amount'] > (q3 + 3 * iqr))
        ).astype(int)

    # 7. Ensure claim_id exists
    if 'claim_id' not in df_clean.columns:
        df_clean['claim_id'] = range(1, len(df_clean) + 1)

    # 8. Drop duplicate claims
    df_clean = df_clean.drop_duplicates(subset=['claim_id'], keep='first')

    print(f"Claims Cleaning Summary -> original: {len(df):,}, cleaned: {len(df_clean):,}")
    return df_clean


def clean_membership_data(df: pd.DataFrame) -> pd.DataFrame:
    """Mirror the training-time membership cleaning."""
    df_clean = df.copy()

    # 1. Standardise column names
    df_clean.columns = df_clean.columns.str.strip().str.lower().str.replace(' ', '_')

    # 2. Parse date columns
    date_columns = [
        'contract_staart_date', 'contract_end_date', 'original_date_of_joining',
        'sscheme_policy_joining_date', 'lapse_date'
    ]
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

    # 3. Filter invalid birth years
    current_year = datetime.now().year
    if 'year_of_birth' in df_clean.columns:
        df_clean = df_clean[(df_clean['year_of_birth'] >= 1920) & (df_clean['year_of_birth'] <= current_year)]
    if 'claimant_year_of_birth' in df_clean.columns:
        df_clean = df_clean[(df_clean['claimant_year_of_birth'] >= 1920) & (df_clean['claimant_year_of_birth'] <= current_year)]

    # 4. Standardise gender
    if 'gender' in df_clean.columns:
        gender_mapping = {
            'M': 'Male', 'F': 'Female', 'O': 'Other',
            'male': 'Male', 'female': 'Female', 'other': 'Other',
            'MALE': 'Male', 'FEMALE': 'Female', 'OTHER': 'Other'
        }
        df_clean['gender'] = df_clean['gender'].map(
            lambda x: gender_mapping.get(x, x) if pd.notna(x) else 'Unknown'
        )

    # 5. Status fields
    if 'status_of_member' in df_clean.columns:
        df_clean['status_of_member'] = df_clean['status_of_member'].astype(str).str.strip().str.title().fillna('Active')
    if 'registration_status' in df_clean.columns:
        df_clean['registration_status'] = df_clean['registration_status'].astype(str).str.strip().str.title().fillna('Registered')
    if 'status_of_registration' in df_clean.columns:
        df_clean['status_of_registration'] = df_clean['status_of_registration'].astype(str).str.strip().str.title()

    # 6. Postcode cleaning
    if 'short_post_code' in df_clean.columns:
        df_clean['short_post_code'] = df_clean['short_post_code'].astype(str).str.strip().str.upper()
    if 'short_post_code_of_member' in df_clean.columns:
        df_clean['short_post_code_of_member'] = df_clean['short_post_code_of_member'].astype(str).str.strip().str.upper()

    # 7. Drop duplicates on unique ID
    if 'unique_id' in df_clean.columns:
        df_clean = df_clean.drop_duplicates(subset=['unique_id'], keep='first')
    elif 'claimant_unique_id' in df_clean.columns:
        df_clean = df_clean.drop_duplicates(subset=['claimant_unique_id'], keep='first')

    print(f"Membership Cleaning Summary -> original: {len(df):,}, cleaned: {len(df_clean):,}")
    return df_clean


In [47]:
def safe_qcut(series: pd.Series, q: int = 5, labels=None):
    """Safely apply qcut even when unique values < q."""
    if labels is None:
        labels = [1, 2, 3, 4, 5]
    series = series.fillna(0)
    unique_vals = series.nunique()
    if unique_vals == 0:
        return pd.Series([1] * len(series), index=series.index)
    q_eff = min(q, unique_vals)
    labels_eff = labels[:q_eff]
    try:
        return pd.qcut(series.rank(method='first'), q=q_eff, labels=labels_eff, duplicates='drop')
    except ValueError:
        return pd.Series([labels_eff[0]] * len(series), index=series.index)


def create_member_features(members_df: pd.DataFrame) -> pd.DataFrame:
    """Copy of the training-time member feature builder."""
    df = members_df.copy()
    current_date = pd.Timestamp.now()

    # Demographics
    if 'year_of_birth' in df.columns:
        df['current_age'] = current_date.year - df['year_of_birth']
        df['age_band'] = pd.cut(df['current_age'], bins=[0, 18, 30, 40, 50, 60, 70, 120],
                                labels=['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '70+'])
        df['age_risk_category'] = pd.cut(df['current_age'], bins=[0, 35, 55, 120],
                                         labels=['Low', 'Medium', 'High'])
    if 'gender' in df.columns:
        df['gender_encoded'] = df['gender'].map({'Male': 1, 'Female': 2, 'Other': 3, 'Unknown': 0})

    # Tenure metrics
    if 'original_date_of_joining' in df.columns:
        df['membership_tenure_days'] = (current_date - df['original_date_of_joining']).dt.days
        df['membership_tenure_years'] = df['membership_tenure_days'] / 365.25
        df['membership_tenure_months'] = df['membership_tenure_days'] / 30.44
        df['tenure_category'] = pd.cut(df['membership_tenure_years'],
                                       bins=[-np.inf, 1, 3, 5, 10, np.inf],
                                       labels=['New(<1yr)', 'Recent(1-3yr)', 'Established(3-5yr)',
                                               'Mature(5-10yr)', 'Long-term(10+yr)'])
    if 'sscheme_policy_joining_date' in df.columns:
        df['scheme_tenure_days'] = (current_date - df['sscheme_policy_joining_date']).dt.days
        df['scheme_tenure_months'] = df['scheme_tenure_days'] / 30.44

    # Contract information
    if 'contract_staart_date' in df.columns and 'contract_end_date' in df.columns:
        df['contract_duration_days'] = (df['contract_end_date'] - df['contract_staart_date']).dt.days
        df['contract_remaining_days'] = (df['contract_end_date'] - current_date).dt.days
        df['contract_elapsed_ratio'] = (
            (current_date - df['contract_staart_date']).dt.days / df['contract_duration_days']
        ).clip(0, 1)
        df['near_renewal'] = (df['contract_remaining_days'] <= 90).astype(int)
        df['contract_expired'] = (df['contract_remaining_days'] < 0).astype(int)
        df['contract_year'] = df['contract_staart_date'].dt.year
        df['contract_start_month'] = df['contract_staart_date'].dt.month
        df['contract_start_quarter'] = df['contract_staart_date'].dt.quarter

    # Status features
    if 'status_of_member' in df.columns:
        df['is_active_member'] = (df['status_of_member'] == 'Active').astype(int)
    if 'registration_status' in df.columns:
        df['is_registered'] = (df['registration_status'] == 'Registered').astype(int)

    # Geography
    if 'short_post_code_of_member' in df.columns:
        df['postcode_area'] = df['short_post_code_of_member'].str[:2]

    # Derived indicators
    if 'membership_tenure_years' in df.columns:
        df['tenure_risk_bucket'] = pd.cut(df['membership_tenure_years'],
                                          bins=[-np.inf, 1, 3, 5, 10, np.inf],
                                          labels=[1, 2, 3, 4, 5])
    return df


In [48]:
def create_claims_features(claims_df: pd.DataFrame, reference_date: pd.Timestamp | None = None) -> pd.DataFrame:
    """Copy of training-time claim level feature engineering."""
    df = claims_df.copy()
    if reference_date is None:
        reference_date = pd.Timestamp.now()

    # Claim amount transformations
    if 'claim_amount' in df.columns:
        df['claim_amount_log'] = np.log1p(df['claim_amount'])
        df['claim_amount_band'] = pd.cut(
            df['claim_amount'],
            bins=[0, 500, 1_000, 2_500, 5_000, 10_000, np.inf],
            labels=['Very Low', 'Low', 'Medium', 'High', 'Very High', 'Critical']
        )
        high_cost_threshold = df['claim_amount'].quantile(0.90)
        df['is_high_cost_claim'] = (df['claim_amount'] >= high_cost_threshold).astype(int)
        catastrophic_threshold = df['claim_amount'].quantile(0.99)
        df['is_catastrophic_claim'] = (df['claim_amount'] >= catastrophic_threshold).astype(int)
    if 'amount_paid' in df.columns:
        df['amount_paid_log'] = np.log1p(df['amount_paid'])

    # Paid date features
    if 'paid_date' in df.columns:
        df['paid_year'] = df['paid_date'].dt.year
        df['paid_month'] = df['paid_date'].dt.month
        df['paid_quarter'] = df['paid_date'].dt.quarter
        df['paid_day_of_week'] = df['paid_date'].dt.dayofweek
        df['paid_week_of_year'] = df['paid_date'].dt.isocalendar().week
        df['paid_day_of_year'] = df['paid_date'].dt.dayofyear
        df['paid_on_weekend'] = (df['paid_day_of_week'] >= 5).astype(int)
        df['paid_in_winter'] = df['paid_month'].isin([12, 1, 2]).astype(int)
        df['paid_in_flu_season'] = df['paid_month'].isin([10, 11, 12, 1, 2, 3]).astype(int)
        df['days_since_paid'] = (reference_date - df['paid_date']).dt.days
        df['months_since_paid'] = df['days_since_paid'] / 30.44
        df['claim_recency'] = pd.cut(
            df['days_since_paid'],
            bins=[-np.inf, 30, 90, 180, 365, np.inf],
            labels=['Very Recent', 'Recent', 'Moderate', 'Old', 'Very Old']
        )

    # Incurred date features
    if 'incurred_date' in df.columns:
        df['incurred_year'] = df['incurred_date'].dt.year
        df['incurred_month'] = df['incurred_date'].dt.month
        df['incurred_quarter'] = df['incurred_date'].dt.quarter
        df['incurred_day_of_week'] = df['incurred_date'].dt.dayofweek
        df['incurred_week_of_year'] = df['incurred_date'].dt.isocalendar().week
        df['days_since_incurred'] = (reference_date - df['incurred_date']).dt.days

    # Processing time
    if 'incurred_date' in df.columns and 'paid_date' in df.columns:
        df['processing_time_days'] = (df['paid_date'] - df['incurred_date']).dt.days
        df['processing_time_weeks'] = df['processing_time_days'] / 7
        df['processing_speed'] = pd.cut(
            df['processing_time_days'],
            bins=[-np.inf, 7, 30, 90, np.inf],
            labels=['Fast', 'Normal', 'Slow', 'Very Slow']
        )
        df['is_delayed_claim'] = (df['processing_time_days'] > 90).astype(int)

    # Admission / length of stay
    if 'admission_date' in df.columns and 'discharge_date' in df.columns:
        df['length_of_stay_days'] = (df['discharge_date'] - df['admission_date']).dt.days
        df['length_of_stay_category'] = pd.cut(
            df['length_of_stay_days'],
            bins=[-np.inf, 0, 1, 3, 7, 14, np.inf],
            labels=['Day_Case', 'Overnight', 'Short', 'Medium', 'Long', 'Extended']
        )
        df['is_inpatient'] = (df['length_of_stay_days'] > 0).astype(int)
        df['admission_day_of_week'] = df['admission_date'].dt.dayofweek
        df['admitted_on_weekend'] = (df['admission_day_of_week'] >= 5).astype(int)
        df['likely_emergency'] = df['admitted_on_weekend']

    # Condition & treatment descriptors
    if 'condition_category' in df.columns:
        df['condition_category_encoded'] = df['condition_category'].astype('category')
        chronic_keywords = ['diabetes', 'cancer', 'cardiac', 'renal', 'chronic']
        df['is_chronic_condition'] = df['condition_category'].str.lower().str.contains(
            '|'.join(chronic_keywords), na=False
        ).astype(int)
    if 'condition_code' in df.columns:
        df['condition_code_category'] = df['condition_code'].astype(str).str[:3]
    if 'treatment_type' in df.columns:
        df['treatment_category'] = df['treatment_type'].astype('category')
        major_keywords = ['surgery', 'operation', 'transplant', 'bypass']
        df['is_major_treatment'] = df['treatment_type'].str.lower().str.contains(
            '|'.join(major_keywords), na=False
        ).astype(int)
    if 'ancillary_service_type' in df.columns:
        df['has_ancillary_service'] = df['ancillary_service_type'].notna().astype(int)

    # Provider/location patterns
    if 'provider_type' in df.columns:
        df['provider_category'] = df['provider_type'].astype('category')
    if 'treatment_location' in df.columns:
        df['treatment_location_category'] = df['treatment_location'].astype('category')

    return df


In [49]:
def create_aggregated_member_claim_features(
    claims_df: pd.DataFrame,
    members_df: pd.DataFrame,
    observation_date: pd.Timestamp | None = None
) -> pd.DataFrame:
    """Aggregate claims history onto members exactly as in training."""
    if observation_date is None:
        observation_date = pd.Timestamp.now()

    member_id_col = 'claimant_unique_id' if 'claimant_unique_id' in claims_df.columns else 'unique_id'
    historical_claims = claims_df[claims_df.get('paid_date', observation_date) <= observation_date].copy()

    # Member level aggregations
    member_agg = historical_claims.groupby(member_id_col).agg({
        'claim_id': 'count',
        'claim_amount': ['sum', 'mean', 'median', 'std', 'max', 'min']
    }).reset_index()
    member_agg.columns = [member_id_col, 'total_claims', 'total_claim_amount', 'avg_claim_amount',
                          'median_claim_amount', 'std_claim_amount', 'max_claim_amount', 'min_claim_amount']

    periods = {
        '12m': 365,
        '6m': 182,
        '3m': 90
    }
    member_period_aggs = []
    for label, days in periods.items():
        cutoff = observation_date - pd.Timedelta(days=days)
        period_claims = historical_claims[historical_claims['paid_date'] >= cutoff]
        period_agg = period_claims.groupby(member_id_col).agg({
            'claim_id': 'count',
            'claim_amount': 'sum'
        }).reset_index()
        period_agg.columns = [member_id_col, f'claim_amount_{label}_count', f'claim_amount_{label}_sum']
        member_period_aggs.append(period_agg)

    # High cost aggregates
    if 'claim_amount' in historical_claims.columns:
        high_cost_threshold = historical_claims['claim_amount'].quantile(0.90)
        high_cost_claims = historical_claims[historical_claims['claim_amount'] >= high_cost_threshold]
        high_cost_agg = high_cost_claims.groupby(member_id_col).agg({
            'claim_amount': ['count', 'sum', 'mean']
        }).reset_index()
        high_cost_agg.columns = [member_id_col, 'claim_amount_high_cost_count',
                                 'claim_amount_high_cost_sum', 'claim_amount_high_cost_mean']
    else:
        high_cost_agg = pd.DataFrame({member_id_col: []})

    claims_sorted = historical_claims.sort_values([member_id_col, 'paid_date'])
    claims_sorted['days_since_last_claim'] = claims_sorted.groupby(member_id_col)['paid_date'].diff().dt.days
    claim_frequency = claims_sorted.groupby(member_id_col)['days_since_last_claim'].agg([
        ('avg_days_between_claims', 'mean'),
        ('median_days_between_claims', 'median'),
        ('min_days_between_claims', 'min')
    ]).reset_index()

    claims_sorted['claim_order'] = claims_sorted.groupby(member_id_col).cumcount() + 1

    def calculate_trend(group: pd.DataFrame) -> float:
        if len(group) < 2:
            return 0.0
        x = group['claim_order'].values
        y = group['claim_amount'].values
        if len(x) > 1 and np.std(x) > 0:
            corr = np.corrcoef(x, y)[0, 1]
            return float(corr) if not np.isnan(corr) else 0.0
        return 0.0

    claim_trends = claims_sorted.groupby(member_id_col).apply(calculate_trend).reset_index()
    claim_trends.columns = [member_id_col, 'claim_amount_trend']

    last_claim = historical_claims.groupby(member_id_col)['paid_date'].max().reset_index()
    last_claim.columns = [member_id_col, 'last_claim_date']
    last_claim['days_since_last_claim'] = (observation_date - last_claim['last_claim_date']).dt.days
    last_claim['months_since_last_claim'] = last_claim['days_since_last_claim'] / 30.44

    first_claim = historical_claims.groupby(member_id_col)['paid_date'].min().reset_index()
    first_claim.columns = [member_id_col, 'first_claim_date']
    first_claim['days_since_first_claim'] = (observation_date - first_claim['first_claim_date']).dt.days

    if 'condition_category' in historical_claims.columns:
        condition_diversity = historical_claims.groupby(member_id_col)['condition_category'].nunique().reset_index()
        condition_diversity.columns = [member_id_col, 'unique_conditions_count']
        chronic_claims = historical_claims[historical_claims.get('is_chronic_condition', 0) == 1]
        chronic_count = chronic_claims.groupby(member_id_col).size().reset_index(name='chronic_condition_claims')
    else:
        condition_diversity = pd.DataFrame({member_id_col: [], 'unique_conditions_count': []})
        chronic_count = pd.DataFrame({member_id_col: [], 'chronic_condition_claims': []})

    if 'treatment_type' in historical_claims.columns:
        treatment_diversity = historical_claims.groupby(member_id_col)['treatment_type'].nunique().reset_index()
        treatment_diversity.columns = [member_id_col, 'unique_treatments_count']
    else:
        treatment_diversity = pd.DataFrame({member_id_col: [], 'unique_treatments_count': []})

    if 'provider_type' in historical_claims.columns:
        provider_diversity = historical_claims.groupby(member_id_col)['provider_type'].nunique().reset_index()
        provider_diversity.columns = [member_id_col, 'unique_providers_count']
    else:
        provider_diversity = pd.DataFrame({member_id_col: [], 'unique_providers_count': []})

    if 'treatment_location' in historical_claims.columns:
        location_diversity = historical_claims.groupby(member_id_col)['treatment_location'].nunique().reset_index()
        location_diversity.columns = [member_id_col, 'unique_locations_count']
    else:
        location_diversity = pd.DataFrame({member_id_col: [], 'unique_locations_count': []})

    if 'is_inpatient' in historical_claims.columns:
        inpatient_stats = historical_claims.groupby(member_id_col).agg({
            'is_inpatient': ['sum', 'mean']
        }).reset_index()
        inpatient_stats.columns = [member_id_col, 'total_inpatient_claims', 'inpatient_ratio']
    else:
        inpatient_stats = pd.DataFrame({member_id_col: [], 'total_inpatient_claims': [], 'inpatient_ratio': []})

    if 'paid_in_winter' in historical_claims.columns:
        seasonal_stats = historical_claims.groupby(member_id_col).agg({
            'paid_in_winter': 'sum',
            'paid_in_flu_season': 'sum'
        }).reset_index()
        seasonal_stats.columns = [member_id_col, 'winter_claims', 'flu_season_claims']
    else:
        seasonal_stats = pd.DataFrame({member_id_col: [], 'winter_claims': [], 'flu_season_claims': []})

    member_agg['claim_amount_cv'] = member_agg['std_claim_amount'] / (member_agg['avg_claim_amount'] + 1)

    member_features = members_df.copy()
    merge_frames = [member_agg] + member_period_aggs + [
        high_cost_agg, claim_frequency, claim_trends, last_claim, first_claim,
        condition_diversity, chronic_count, treatment_diversity, provider_diversity,
        location_diversity, inpatient_stats, seasonal_stats
    ]
    for frame in merge_frames:
        if not frame.empty:
            member_features = member_features.merge(frame, on=member_id_col, how='left')

    count_columns = [
        'total_claims', 'claim_amount_12m_count', 'claim_amount_6m_count', 'claim_amount_3m_count',
        'claim_amount_high_cost_count', 'winter_claims', 'flu_season_claims',
        'chronic_condition_claims', 'total_inpatient_claims', 'unique_conditions_count',
        'unique_treatments_count', 'unique_providers_count', 'unique_locations_count'
    ]
    for col in count_columns:
        if col in member_features.columns:
            member_features[col] = member_features[col].fillna(0)

    sum_columns = [
        'total_claim_amount', 'claim_amount_12m_sum', 'claim_amount_6m_sum',
        'claim_amount_3m_sum', 'claim_amount_high_cost_sum'
    ]
    for col in sum_columns:
        if col in member_features.columns:
            member_features[col] = member_features[col].fillna(0)

    if 'inpatient_ratio' in member_features.columns:
        member_features['inpatient_ratio'] = member_features['inpatient_ratio'].fillna(0)
    if 'claim_amount_cv' in member_features.columns:
        member_features['claim_amount_cv'] = member_features['claim_amount_cv'].fillna(0)

    if {'claim_amount_3m_count', 'total_claims'}.issubset(member_features.columns):
        member_features['claims_acceleration_3m'] = member_features['claim_amount_3m_count'] - (
            member_features['total_claims'] / ((member_features['membership_tenure_years'] * 12) + 1)
        )
    if {'claim_amount_12m_count', 'membership_tenure_years'}.issubset(member_features.columns):
        member_features['claims_acceleration_12m'] = member_features['claim_amount_12m_count'] - (
            member_features['total_claims'] / (member_features['membership_tenure_years'] + 1)
        )
    if {'claim_amount_3m_sum', 'claim_amount_12m_sum'}.issubset(member_features.columns):
        member_features['recent_activity_ratio'] = member_features['claim_amount_3m_sum'] / (
            member_features['claim_amount_12m_sum'] + 1
        )
    if {'claim_amount_high_cost_sum', 'total_claim_amount'}.issubset(member_features.columns):
        member_features['high_cost_ratio'] = member_features['claim_amount_high_cost_sum'] / (
            member_features['total_claim_amount'] + 1
        )
    if {'total_claims', 'membership_tenure_years'}.issubset(member_features.columns):
        member_features['claims_per_year_of_membership'] = member_features['total_claims'] / (
            member_features['membership_tenure_years'] + 0.1
        )
    if {'avg_claim_amount', 'max_claim_amount'}.issubset(member_features.columns):
        member_features['avg_to_max_ratio'] = member_features['avg_claim_amount'] / (
            member_features['max_claim_amount'] + 1
        )

    def _column_or_zero(df: pd.DataFrame, col: str) -> pd.Series:
        if col in df.columns:
            return df[col].fillna(0)
        return pd.Series(0, index=df.index, dtype=float)

    member_features['has_claim_history'] = (_column_or_zero(member_features, 'total_claims') > 0).astype(int)
    member_features['has_recent_claims'] = (_column_or_zero(member_features, 'claim_amount_3m_count') > 0).astype(int)
    member_features['has_high_cost_history'] = (_column_or_zero(member_features, 'claim_amount_high_cost_count') > 0).astype(int)
    member_features['has_chronic_history'] = (_column_or_zero(member_features, 'chronic_condition_claims') > 0).astype(int)

    total_claims_series = _column_or_zero(member_features, 'total_claims')
    member_features['is_frequent_claimer'] = (total_claims_series > total_claims_series.median()).astype(int)

    member_features['risk_score'] = (
        member_features['has_high_cost_history'] * 2 +
        member_features['has_chronic_history'] +
        member_features['is_frequent_claimer'] +
        member_features['has_recent_claims']
    )
    return member_features


In [50]:
def full_feature_engineering_pipeline(
    claims_df: pd.DataFrame,
    members_df: pd.DataFrame,
    observation_date: pd.Timestamp | None = None
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Run the same feature pipeline used during training."""
    print("=" * 80)
    print("FEATURE ENGINEERING PIPELINE")
    print("=" * 80)

    if observation_date is None:
        observation_date = pd.Timestamp.now()
    print(f"\nObservation Date: {observation_date.date()}")

    print("\n1. Creating member-level features...")
    members_featured = create_member_features(members_df)
    print(f"   Member features created. Shape: {members_featured.shape}")

    print("\n2. Creating claim-level features...")
    claims_featured = create_claims_features(claims_df, reference_date=observation_date)
    print(f"   Claim features created. Shape: {claims_featured.shape}")

    print("\n3. Creating aggregated member-claim features...")
    final_dataset = create_aggregated_member_claim_features(claims_featured, members_featured, observation_date)
    print(f"   Aggregated features created. Shape: {final_dataset.shape}")

    print("\n" + "=" * 80)
    print("FEATURE ENGINEERING COMPLETE")
    print(f"Final Dataset Shape: {final_dataset.shape}")
    print(f"Total Features: {len(final_dataset.columns)}")
    print("=" * 80)

    return final_dataset, claims_featured


In [51]:
def prepare_inference_features(
    raw_claims: pd.DataFrame,
    raw_members: pd.DataFrame,
    observation_date: pd.Timestamp | None = None
) -> tuple[pd.DataFrame, pd.Timestamp]:
    """Clean inputs and rebuild feature matrix for inference."""
    claims_clean = clean_claims_data(raw_claims)
    members_clean = clean_membership_data(raw_members)

    # Choose observation date as latest paid date if not provided
    if observation_date is None:
        if 'paid_date' in claims_clean.columns and claims_clean['paid_date'].notna().any():
            observation_date = claims_clean['paid_date'].max()
        else:
            observation_date = pd.Timestamp.now()

    final_memberset, _ = full_feature_engineering_pipeline(
        claims_clean, members_clean, observation_date
    )

    if 'scheme_type' in final_memberset.columns:
        final_memberset = final_memberset.drop(columns=['scheme_type'])

    return final_memberset, observation_date


In [58]:
# --- 1. Ensure input dataframes are available ---------------------------------
try:
    claims_input = claims_sample.copy()
except NameError as exc:
    raise RuntimeError("Define 'claims_sample' before running inference.") from exc

try:
    members_input = mem_combined.copy()
except NameError:
    try:
        members_input = mem_combine.copy()
    except NameError as exc:
        raise RuntimeError("Define 'mem_combined' (or 'mem_combine') before running inference.") from exc

# --- 2. Rebuild features -------------------------------------------------------
member_feature_df, as_of_date = prepare_inference_features(claims_input, members_input)
member_id_col = 'claimant_unique_id' if 'claimant_unique_id' in member_feature_df.columns else 'unique_id'
if member_id_col not in member_feature_df.columns:
    raise ValueError("Could not identify member identifier column in engineered features.")

# --- 3. Load production artifacts ---------------------------------------------
model_dir = 'production_models'
if not os.path.isdir(model_dir):
    raise FileNotFoundError("production_models directory not found. Run the training notebook export step first.")

paths = {
    'regressor': os.path.join(model_dir, 'xgb_regressor.pkl'),
    'classifier': os.path.join(model_dir, 'xgb_classifier.pkl'),
    'multiclass': os.path.join(model_dir, 'xgb_multiclass.pkl'),
    'scaler': os.path.join(model_dir, 'scaler.pkl'),
    'encoders': os.path.join(model_dir, 'label_encoders.pkl'),
    'risk_encoder': os.path.join(model_dir, 'risk_encoder.pkl'),
    'feature_names': os.path.join(model_dir, 'feature_names.json'),
    'metadata': os.path.join(model_dir, 'metadata.json')
}

missing = [name for name, path in paths.items() if name in ['regressor', 'classifier', 'feature_names'] and not os.path.exists(path)]
if missing:
    raise FileNotFoundError(f"Missing required artifact(s): {missing}. Re-run model export in the training notebook.")

reg_model = joblib.load(paths['regressor'])
clf_model = joblib.load(paths['classifier'])
multiclass_model = joblib.load(paths['multiclass']) if os.path.exists(paths['multiclass']) else None
scaler = joblib.load(paths['scaler']) if os.path.exists(paths['scaler']) else None
label_encoders = joblib.load(paths['encoders']) if os.path.exists(paths['encoders']) else {}
risk_encoder = joblib.load(paths['risk_encoder']) if os.path.exists(paths['risk_encoder']) else None

with open(paths['feature_names'], 'r') as f:
    feature_names = json.load(f)
metadata = {}
if os.path.exists(paths['metadata']):
    with open(paths['metadata'], 'r') as f:
        metadata = json.load(f)

# --- 4. Align feature matrix ---------------------------------------------------
features = member_feature_df.copy()
for col in feature_names:
    if col not in features.columns:
        features[col] = 0

missing_after = [col for col in feature_names if col not in features.columns]
if missing_after:
    raise ValueError(f"Unable to construct required features: {missing_after}")

X = features[feature_names].copy()

for col, encoder in label_encoders.items():
    if col in X.columns:
        mapping = {cls: idx for idx, cls in enumerate(encoder.classes_)}
        X[col] = X[col].astype(str).map(mapping).fillna(-1).astype(int)

X = X.replace([np.inf, -np.inf], np.nan).fillna(-999)
X_matrix = scaler.transform(X) if scaler is not None else X.values

# --- 5. Run predictions --------------------------------------------------------
reg_predictions = reg_model.predict(X_matrix)
clf_probabilities = clf_model.predict_proba(X_matrix)[:, 1]
clf_flags = (clf_probabilities >= 0.5).astype(int)

if multiclass_model is not None and risk_encoder is not None:
    risk_labels = risk_encoder.inverse_transform(multiclass_model.predict(X_matrix))
else:
    risk_labels = None

# --- 6. Assemble output -------------------------------------------------------
results = pd.DataFrame({
    member_id_col: member_feature_df[member_id_col],
    'Predicted_Claim_Amount': reg_predictions,
    'High_Claim_Probability': clf_probabilities,
    'Will_Have_High_Claim': clf_flags
})
if risk_labels is not None:
    results['Predicted_Risk_Category'] = risk_labels

if 'high_claim_threshold' in metadata:
    results.attrs['high_claim_threshold'] = metadata['high_claim_threshold']

results_sorted = results.sort_values('High_Claim_Probability', ascending=False).reset_index(drop=True)

print(f"Inference completed for {len(results_sorted):,} members as of {as_of_date.date()}.")
results_sorted.head()


C:\Temp\ipykernel_26072\968998591.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['treatment_type'].fillna('Unknown', inplace=True)
C:\Temp\ipykernel_26072\968998591.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

Claims Cleaning Summary -> original: 1,000, cleaned: 992
Membership Cleaning Summary -> original: 873, cleaned: 873
FEATURE ENGINEERING PIPELINE

Observation Date: 2025-11-02

1. Creating member-level features...
   Member features created. Shape: (873, 29)

2. Creating claim-level features...
   Claim features created. Shape: (992, 70)

3. Creating aggregated member-claim features...


C:\Temp\ipykernel_26072\1216600732.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  claim_trends = claims_sorted.groupby(member_id_col).apply(calculate_trend).reset_index()


   Aggregated features created. Shape: (873, 75)

FEATURE ENGINEERING COMPLETE
Final Dataset Shape: (873, 75)
Total Features: 75


c:\Users\sumit\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Inference completed for 873 members as of 2025-11-02.


c:\Users\sumit\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


,claimant_unique_id,Predicted_Claim_Amount,High_Claim_Probability,Will_Have_High_Claim
0,MEM00014922-01,104851.774649,0.722488,1
1,MEM00004162-01,21911.361439,0.148439,0
2,MEM00003152-02,21327.256457,0.104796,0
3,MEM00012511-02,21368.722743,0.088494,0
4,MEM00048133-03,25964.177571,0.029278,0


In [88]:
results_sorted[results_sorted['Will_Have_High_Claim'] == 1]

,claimant_unique_id,Predicted_Claim_Amount,High_Claim_Probability,Will_Have_High_Claim
0,MEM00014922-01,104851.774649,0.722488,1


In [ ]:
from datetime import datetime

In [ ]:
# Optional: persist outputs for downstream use
artifacts_dir = 'artifacts'
os.makedirs(artifacts_dir, exist_ok=True)

predictions_path = os.path.join(artifacts_dir, 'production_inference_predictions.csv')
results_sorted.to_csv(predictions_path, index=False)
print(f"Saved member-level predictions to: {predictions_path}")


In [35]:
import pandas as pd
import numpy as np

In [3]:
claims_df = pd.read_csv("uk_pmi_claims_200k.csv")
memebrship_df = pd.read_csv("uk_pmi_membership_120k.csv")

In [5]:
claims_sample = claims_df.sample(1000)

In [11]:
a = set(claims_sample['Claimant Unique ID'])

In [16]:
x = memebrship_df[memebrship_df['Unique ID'].isin(a)]

In [17]:
y = memebrship_df[~memebrship_df['Unique ID'].isin(a)].sample(100)

In [40]:
mem_combined = pd.concat([x,y], ignore_index=True)

In [43]:
mem_combined = mem_combined.rename(columns = {'Unique ID':'claimant_unique_id'})

In [44]:
mem_combined.sample(5)

,Client Name,Client Identifier,Scheme Category/ Section Name,Scheme Category/ Section Name Identifier,Unique Member Reference,claimant_unique_id,Status of Member,Status of Registration,Year of Birth,Gender,Short Post Code of Member,Contract Start Date,Contract End Date,Original Date of Joining,Scheme Policy Joining Date,Registration Status,Lapse Date
268,Medicash,CLI0014,Comprehensive Cover,SCH0002,MEM00018476,MEM00018476-02,Partner,Family,1994,Male,B16,2022-10-31,2023-10-31,2020-10-15,2022-05-23,Active,NaN
286,Healix Health Services,CLI0009,Essential Cover,SCH0009,MEM00019592,MEM00019592-01,Member,Single,1961,Female,RG2,2023-01-15,2024-01-15,2021-06-19,2022-08-14,Active,NaN
497,HSF Health Plan,CLI0012,Core Cover,SCH0001,MEM00033757,MEM00033757-01,Member,Family,1991,Female,CR0,2023-04-23,2024-04-22,2022-12-24,2022-06-14,Active,NaN
78,Benenden Health,CLI0007,Elite Plan,SCH0012,MEM00005666,MEM00005666-01,Member,Family,1967,Male,M1,2020-12-11,2021-12-11,2018-10-03,2020-01-08,Active,NaN
763,Westfield Health,CLI0013,Platinum Plan,SCH0008,MEM00052440,MEM00052440-03,Dependent,Couple,2005,Female,LS1,2023-10-25,2024-10-24,2021-11-27,2023-04-04,Active,NaN
